In [95]:
import os
import matplotlib
import numpy as np
import pandas as pd
import csv
import datetime
from collections import Counter

pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', None) 
pd.options.plotting.backend = "plotly"

In [2]:
directory = "API-logs/"

### Code Carbon logs (18 hours for now)

In [278]:
carbondf = pd.DataFrame(columns = ['timestamp', 'RAM energy', 'GPU energy', 'CPU energy', 'Energy consumed'])

In [279]:
with open(directory+'carbon.log' , 'r') as f:
    lines = f.readlines() # readlines creates a list of the lines
len(lines)

27952

In [280]:
for i in range(0, len(lines), 4):
    for line in lines[i:i + 4]:
        try:
            time = line.split('@')[1].split(']')[0].strip()
        except:
            continue
        if 'Energy consumed for RAM :' in line: 
            ram_nrj = line.split('Energy consumed for RAM :')[1].split('RAM')[0].replace('kWh.','')
        elif 'Energy consumed for all GPUs :'in line:
            gpu_nrj = line.split('Energy consumed for all GPUs :')[1].split('All GPUs')[0].replace('kWh.','')
        elif 'Energy consumed for all CPUs :' in line:
            cpu_nrj= line.split('Energy consumed for all CPUs :')[1].split('All CPUs')[0].replace('kWh.','')
        elif ' electricity used since the begining.' in line:
            total_nrj = line.split(' electricity used since the begining.')[0].split(']')[1].replace(' of','').replace('kWh','')
        carbondf.loc[i] = [time, ram_nrj, gpu_nrj, cpu_nrj, total_nrj]
carbondf.shape       

(6933, 5)

In [281]:
carbondf['timestamp'] = pd.to_datetime(carbondf['timestamp'], format= "%H:%M:%S")
#.dt.time

In [282]:
carbondf.shape

(6933, 5)

In [283]:
carbondf.head()

,timestamp,RAM energy,GPU energy,CPU energy,Energy consumed
0,1900-01-01 15:12:02,0.001395,0.006703,0.000125,0.008222
4,1900-01-01 15:12:12,0.002708,0.013103,0.000243,0.016054
8,1900-01-01 15:12:22,0.004022,0.021320,0.000361,0.025703
12,1900-01-01 15:12:32,0.005333,0.024947,0.000478,0.030758
16,1900-01-01 15:12:42,0.006658,0.028631,0.000597,0.035886


In [286]:
#this isn't working!
carbonhourdf = carbondf.groupby(pd.Grouper(key = 'timestamp', freq='60min'))
carbonhourdf.head()

,timestamp,RAM energy,GPU energy,CPU energy,Energy consumed
13404,1900-01-01 00:00:03,4.177790,17.623274,0.374012,22.173751
13408,1900-01-01 00:00:13,4.179115,17.626703,0.374130,22.178623
13412,1900-01-01 00:00:23,4.180438,17.630131,0.374248,22.183494
13416,1900-01-01 00:00:33,4.181761,17.633568,0.374366,22.188373
13420,1900-01-01 00:00:43,4.183085,17.636999,0.374484,22.193244
14928,1900-01-01 01:00:03,4.653188,19.090999,0.416513,24.159382
14932,1900-01-01 01:00:13,4.654493,19.094413,0.416632,24.164233
14936,1900-01-01 01:00:23,4.655815,19.097790,0.416749,24.169032
14940,1900-01-01 01:00:33,4.657138,19.101215,0.416867,24.173897
14944,1900-01-01 01:00:43,4.658462,19.104640,0.416985,24.178763


### Server logs

In [304]:
with open(directory+'time-server.log','r') as f:
    lines = f.readlines() # readlines creates a list of the lines
len(lines)

13384

In [310]:
requestlist=[]
for line in lines:
    time = datetime.datetime.strptime(line.replace('\n',''), "%d/%b/%Y %H:%M:%S")
    requestlist.append(time)
len(requestlist)

13384

In [311]:
counts = Counter(h.hour for h in requestlist)
hour_count = [{"hour": h, "count": c} for h, c in counts.items()]
print(hour_count)

[{'hour': 15, 'count': 1338}, {'hour': 16, 'count': 1609}, {'hour': 17, 'count': 1426}, {'hour': 18, 'count': 1544}, {'hour': 19, 'count': 1580}, {'hour': 20, 'count': 1006}, {'hour': 21, 'count': 420}, {'hour': 22, 'count': 448}, {'hour': 23, 'count': 608}, {'hour': 0, 'count': 446}, {'hour': 1, 'count': 232}, {'hour': 2, 'count': 401}, {'hour': 3, 'count': 176}, {'hour': 4, 'count': 352}, {'hour': 5, 'count': 364}, {'hour': 6, 'count': 408}, {'hour': 7, 'count': 470}, {'hour': 8, 'count': 320}, {'hour': 9, 'count': 236}]


### TODO: 

1) calculate energy consumption by hour

2) plot it with the number of requests by hour

3) look at refused requests:

200 = OK we answered

400 = used bad request